### Übung 04 - Bewegungsartefakte

## Hinweis

In einigen Aufgaben wählen sie selbsständig Werte um Funktionen aufzurufen. Sollten sie nach Abschluss der Aufgaben noch Zeit haben, nutzen sie die Gelegenheit und probieren sie die Werte interaktiv zu setzen um zum Beispiel mit Schiebereglern ein besseres Verständnis für Auswirkungen und Nebeneffekte von verschiedenen Eingabewerten zu erhalten.

[ipywidgets](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html#Numeric-widgets)

In [2]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import colors

In [3]:
# platzieren Sie die Datei "phantom_256x256.npy" im selben Ordner wie dieses Notebook, bevor Sie diese Zelle ausführen
plt.rcParams["figure.figsize"] = (20,10)
kspace = np.load('phantom_256x256.npy')
image = np.fft.ifftshift(np.fft.ifft2(np.fft.ifftshift(kspace)))

## Aufgabe 1 - Translationsbewegung

Eine Translationsbewegung ist eine Verschiebung eines Objektes im Raum. Das heißt, das zu scannende Objekt wird zum Beispiel in x- oder y-Richtung verschoben. Simulieren Sie eine Translationsbewegung, indem Sie das Objekt im rekonstruierten Bild (in der Variable `image`) verschieden weit in x- und/oder -y Richtung verschieben.
Transformieren Sie das verschobene Bild dann wieder in den k-Raum. Probieren Sie das ganze für verschieden starke Translationsbewegungen und -richtungen aus und beobachten Sie den Effekt der Translationsbewegung auf den k-Raum (Betrag und Phase).

Tipp 1: Definieren Sie sich zunächst eine Funktion, die die Transformation in den k-Raum durchführt. Testen Sie die Funktion, indem Sie das Ergebnis Ihrer Funktion mit dem ursprünglichen k-Raum (in der Variable `kspace`) vergleichen.

Tipp 2: Für die Translationsbewegung könnte die Funktion `numpy.roll` helfen.

Tipp 3: Nutzen Sie beim k-Raum für die Visualisierung immer eine logarithmische Skala für den Betrag und eine lineare Skala für die Phase.

Tipp 4: Es kann für die Beobachtung helfen Slider für die x- und y-Verschiebung zu verwenden (über ipywidgets)

Wie verändert die Translationsbewegung den k-Raum? Überlegen Sie sich außerdem, warum das so ist.

<span style="color:blue">(TODO - Schreiben Sie Ihre Antwort hier hin!) </span>

## Aufgabe 2 - Translationsbewegung während der Aufnahme (y-Richtung)

Simulieren Sie nun, wie sich eine Translationsbewegung bei einer MRT-Aufnahme auswirkt. Das Objekt soll sich mit jeder vierten aufgenommenen k-Raum-Zeile um einen Pixel in y-Richtung bewegen. Sobald es sich um insgesamt 5 Pixel bewegt hat, soll es sich wieder in negative y-Richtung bewegen, bis es bei -5 Pixeln von der Ausgangsposition angekommen ist. Dann soll es wieder die Richtung ändern, bis es bei +5 Pixeln von der Ausgangsposition angekommen ist, usw. bis der k-Raum gefüllt ist.
Rekonstruieren Sie dann das Bild, aus dem auf diese Weise generierten k-Raum und visualisieren Sie es.

Hinweis: Die Phasenkodierung erfolgt in y-Richtung

Wie zeigt sich die Bewegung im Bild?

<span style="color:blue">(TODO - Schreiben Sie Ihre Antwort hier hin!) </span>

## Aufgabe 3 - Translationsbewegung während der Aufnahme (x-Richtung)

Führen Sie das gleiche "Experiment" wie in der vorherigen Aufgabe durch nur, dass diesmal die Bewegung in x-Richtung passieren soll.

Wie zeigt sich die Bewegung nun? Vergleichen Sie dieses Bild mit dem aus der letzten Aufgabe

<span style="color:blue">(TODO - Schreiben Sie Ihre Antwort hier hin!) </span>

## Aufgabe 4

Wir nehmen nun an, dass die Bewegung nur in einer Hälfte der Zeilen auftritt, und zwar einmal während der Mitte (Zeilen 64 bis 191) und einmal am Anfang & Ende (Zeilen 0 bis 63 und 192 bis 255). Simulieren Sie beide Szenarien und visualisieren Sie die entstehenden Bilder.

Was fällt auf?

<span style="color:blue">(TODO - Schreiben Sie Ihre Antwort hier hin!) </span>

## Aufgabe 5 - Rotationsbewegung

In dieser Aufgabe werden wir den Einfluss von Rotationsbewegungen auf den k-Raum untersuchen. Simulieren Sie daher ähnlich wie in Aufgabe 1 ein Rotationsbewegung, indem Sie das Objekt im Bildraum um verschiedene Winkel rotieren und den Effekt auf den k-Raum beobachten.

Tipp: Für die Rotation lohnt sich ein Blick auf die Funktion `scipy.ndimage.rotate`


Beschreiben Sie, wie sich die Rotationsbewegung auswirkt und überlegen Sie, warum das so sein könnte.

<span style="color:blue">(TODO - Schreiben Sie Ihre Antwort hier hin!) </span>

## Aufgabe 6 - Rotationsbewegung während der Aufnahme

Ähnlich wie in den Aufgaben 2 und 3 soll nun simuliert werden, dass das Objekt sich während der Aufnahme bewegt. Das Objekt soll alternierend zwischen +2° und -2° vom Ausgangszustand rotiert werden. Mit jeder aufgenommenen Zeile, soll es um 2° (bzw. -2°) rotiert werden.
Visualisieren Sie das dabei entstandene Bild.

Beschreiben Sie, was auf dem Bild zu sehen ist. Wie ist das zu erklären?

<span style="color:blue">(TODO - Schreiben Sie Ihre Antwort hier hin!) </span>